In [1]:
1+1

2

In [2]:
import os
from dotenv import load_dotenv
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

class AddInput(BaseModel):
    x: str = Field(description="첫 번째 숫자")
    y: str = Field(description="두 번째 숫자")

class PopulationInput(BaseModel):
    city: str = Field(description="인구를 조회할 도시명")

def add(x: str, y: str) -> str:
    print(f"x:{x}, y:{y}")
    try:
        return str(float(x) + float(y))
    except Exception:
        return "계산 실패"


def lookup_population(city: str) -> str:
    print(f"city:{city}")

    population = {
        "서울": "900만",
        "부산": "300만",
    }
    return population.get(city.strip(), "정보 없음")


tools = [
    StructuredTool.from_function(
        name="Add",
        func=add,
        description="두 숫자를 더합니다.",
        args_schema=AddInput
    ),
    StructuredTool.from_function(
        name="PopulationLookup",
        func=lookup_population,
        description="도시명을 입력하면 인구를 반환합니다.",
        args_schema=PopulationInput,
    ),
]

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    verbose=True,
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

print("OpenAI Functions 실행:")
print(agent.run("5에 2를 더하고, 서울 인구도 알려줘"))





OpenAI Functions 실행:


> Entering new AgentExecutor chain...

Invoking: `Add` with `{'x': '5', 'y': '2'}`


x:5, y:2
7.0
Invoking: `PopulationLookup` with `{'city': '서울'}`


city:서울
900만5에 2를 더하면 7입니다. 서울의 인구는 약 900만입니다.

> Finished chain.
5에 2를 더하면 7입니다. 서울의 인구는 약 900만입니다.
